In [1]:
##########################
#### standard library ####
##########################
import os
import copy
import time
import gc
from itertools import  filterfalse as ifilterfalse
import warnings
import random
from typing import List, Tuple, Dict, Any, Union, Optional, Sequence, Iterable, Callable
import shutil
# warnings.filterwarnings("ignore")

###################
#### 3rd party ####
###################
import torch
import torchaudio
import pandas as pd
from tqdm import tqdm
import soundfile as sf

######################
#### my own files ####
######################
pass

In [2]:
def read_wav(path, new_freq): # pip install sox       and      sudo apt-get install libsox-dev
    wav, org_sr = torchaudio.load(path, normalize=True)
    wav = torchaudio.functional.resample(wav, orig_freq=org_sr, new_freq=new_freq)
    return wav


def crop_start_wav(wav, duration_):
    while wav.size(-1) < duration_:
        wav = torch.cat([wav, wav], dim=1)
    wav = wav[:, :duration_]
    return wav


In [7]:


def remake_data_smaller(
        data_dir:str = "/home/nikita/hdd/Data/BirdCLEF2024",
        s:float = 5, #seconds
        sample_rate:int = 32000, #Hz
        out_dir:str = "/home/nikita/hdd/Data/BirdCLEF2024_5s_32000Hz",
    ):
    # create new dir and copy metadata
    os.makedirs(f"{out_dir}/train_audio", exist_ok=True)
    shutil.copy(f"{data_dir}/train_metadata.csv", f"{out_dir}/train_metadata.csv")
    shutil.copy(f"{data_dir}/sample_submission.csv", f"{out_dir}/sample_submission.csv")

    # read metadata
    df = pd.read_csv(f"{out_dir}/train_metadata.csv")
    df["in_path"] = data_dir + "/train_audio/" + df["filename"]
    df["out_path"] = out_dir + "/train_audio/" + df["filename"]

    # remake data
    for i, row in tqdm(df.iterrows(), total=len(df)):
        wav = read_wav(row["in_path"], sample_rate)
        wav = crop_start_wav(wav, s*sample_rate).squeeze().numpy()
        destination_folder = os.path.dirname(row["out_path"])
        os.makedirs(destination_folder, exist_ok=True)
        sf.write(row["out_path"], wav, sample_rate, format="ogg", )

remake_data_smaller()

100%|██████████| 24459/24459 [52:27<00:00,  7.77it/s]  
